In [1]:
import pandas as pd
import pickle
import numpy as np
#from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [5]:
df_1=pd.read_csv('ED_3hour_foward_full.csv')

In [109]:
name="label_after_2hour"

In [110]:
#df_1=df_1[~df_1['stay_id'].isin(df_1[df_1[name]==2]['stay_id'].unique())]

In [111]:
#df_1.loc[df_1[name] == 1, name] = 0
#df_1.loc[df_1[name] == 2, name] = 0
#df_1.loc[df_1[name] == 3, name] = 1

In [112]:
from sklearn.model_selection import train_test_split

icu_samples = df_1[df_1[name] == 3]
hos_samples = df_1[df_1[name] == 2]

icu_stay_id = icu_samples['stay_id'].unique()

train_icu_stay_id, icu_temp_stay_id = train_test_split(icu_stay_id, test_size=0.2, random_state=42)
test_icu_stay_id, val_icu_stay_id = train_test_split(icu_temp_stay_id, test_size=0.2, random_state=42)

train_icu_data = df_1[df_1['stay_id'].isin(train_icu_stay_id)]
test_icu_data = df_1[df_1['stay_id'].isin(test_icu_stay_id)]
val_icu_data = df_1[df_1['stay_id'].isin(val_icu_stay_id)]

hos_stay_id = hos_samples['stay_id'].unique()

train_hos_stay_id, hos_temp_stay_id = train_test_split(hos_stay_id, test_size=0.2, random_state=42)
test_hos_stay_id, val_hos_stay_id = train_test_split(hos_temp_stay_id, test_size=0.2, random_state=42)

train_hos_data = df_1[df_1['stay_id'].isin(train_hos_stay_id)]
test_hos_data = df_1[df_1['stay_id'].isin(test_hos_stay_id)]
val_hos_data = df_1[df_1['stay_id'].isin(val_hos_stay_id)]

ed_data=df_1[~df_1['stay_id'].isin(list(icu_stay_id)+list(hos_stay_id))]

train_ed_stay_id, ed_temp_stay_id = train_test_split(ed_data['stay_id'].unique(), test_size=0.2, random_state=42)
test_ed_stay_id, val_ed_stay_id = train_test_split(ed_temp_stay_id, test_size=0.2, random_state=42)

train_ed_data = df_1[df_1['stay_id'].isin(train_ed_stay_id)]
test_ed_data = df_1[df_1['stay_id'].isin(test_ed_stay_id)]
val_ed_data = df_1[df_1['stay_id'].isin(val_ed_stay_id)]

train_df = pd.concat([train_icu_data, train_hos_data, train_ed_data ])
test_df = pd.concat([test_icu_data, test_hos_data, test_ed_data])
val_df = pd.concat([val_icu_data, val_hos_data, val_ed_data])

In [113]:
#list(icu_stay_id)+list(hos_stay_id)

In [114]:
train_df=train_df.sort_index()
test_df=test_df.sort_index()
val_df=val_df.sort_index()

In [115]:
train_df

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,...,label_after_6hour,gender_F,gender_M,race_Black,race_Other,race_White,arrival_transport_AMBULANCE,arrival_transport_HELICOPTER,arrival_transport_OTHER,arrival_transport_WALK IN
0,30000012.0,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,...,2.0,1,0,0,0,1,1,0,0,0
1,30000012.0,2126-02-14 22:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,...,2.0,1,0,0,0,1,1,0,0,0
2,30000012.0,2126-02-14 23:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,...,2.0,1,0,0,0,1,1,0,0,0
3,30000012.0,2126-02-15 00:22:00,37.111111,80.0,13.0,99.0,112.0,44.0,37.111111,80.0,...,2.0,1,0,0,0,1,1,0,0,0
4,30000012.0,2126-02-15 01:22:00,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,80.0,...,2.0,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7149067,39999835.0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0,0,0,0,0,0,0,0,0
7149068,39999835.0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0,0,0,0,0,0,0,0,0
7149069,39999835.0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0,0,0,0,0,0,0,0,0
7149070,39999835.0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0,0,0,0,0,0,0,0,0


In [116]:
print(len(train_df['stay_id'].unique()))
print(len(test_df['stay_id'].unique()))
print(len(val_df['stay_id'].unique()))

238304
47661
11916


In [117]:
train_x=train_df.drop(['stay_id','time','label_after_1hour','label_after_2hour','label_after_3hour','label_after_4hour','label_after_5hour','label_after_6hour'],axis='columns')
train_y=train_df[name]
test_x=test_df.drop(['stay_id','time','label_after_1hour','label_after_2hour','label_after_3hour','label_after_4hour','label_after_5hour','label_after_6hour'],axis='columns')
test_y=test_df[name]
val_x=val_df.drop(['stay_id','time','label_after_1hour','label_after_2hour','label_after_3hour','label_after_4hour','label_after_5hour','label_after_6hour'],axis='columns')
val_y=val_df[name]

In [118]:
train_x = train_x.astype(np.float32)
train_y = train_y.astype(np.float32)
test_x = test_x.astype(np.float32)
test_y = test_y.astype(np.float32)
val_x = val_x.astype(np.float32)
val_y = val_y.astype(np.float32)
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()
train_data_x_reshaped = train_x.reshape((len(train_df['stay_id'].unique()), 24, 114))
train_data_y_reshaped = train_y.reshape((len(train_df['stay_id'].unique()), 24,1))
test_data_x_reshaped = test_x.reshape((len(test_df['stay_id'].unique()), 24, 114))
test_data_y_reshaped = test_y.reshape((len(test_df['stay_id'].unique()), 24, 1))
val_x_reshaped = val_x.reshape((len(val_df['stay_id'].unique()), 24, 114))
val_y_reshaped = val_y.reshape((len(val_df['stay_id'].unique()), 24, 1))

In [119]:
train_data_y_reshaped.shape

(238304, 24, 1)

In [120]:
largest_values = np.max(train_data_y_reshaped, axis=1, keepdims=True) 
train_data_y_reshaped_1 = largest_values.reshape(len(train_df['stay_id'].unique()), 1, 1)

largest_values = np.max(test_data_y_reshaped, axis=1, keepdims=True) 
test_data_y_reshaped_1 = largest_values.reshape(len(test_df['stay_id'].unique()), 1, 1)

largest_values = np.max(val_y_reshaped, axis=1, keepdims=True) 
val_y_reshaped_1 = largest_values.reshape(len(val_df['stay_id'].unique()), 1, 1)

In [121]:
from keras.utils import to_categorical

# Assuming you have three classes
num_classes = 4

# For train data
largest_values_train = np.max(train_data_y_reshaped, axis=1, keepdims=True)
train_data_y_reshaped_1 = to_categorical(largest_values_train, num_classes=num_classes)

# For test data
largest_values_test = np.max(test_data_y_reshaped, axis=1, keepdims=True)
test_data_y_reshaped_1 = to_categorical(largest_values_test, num_classes=num_classes)

# For validation data
largest_values_val = np.max(val_y_reshaped, axis=1, keepdims=True)
val_y_reshaped_1 = to_categorical(largest_values_val, num_classes=num_classes)

In [122]:
train_data_y_reshaped_1=train_data_y_reshaped_1.reshape(-1,4)
test_data_y_reshaped_1=test_data_y_reshaped_1.reshape(-1,4)
val_y_reshaped_1=val_y_reshaped_1.reshape(-1,4)

In [123]:
train_data_x_reshaped.shape

(238304, 24, 114)

In [124]:
train_dataset = TensorDataset(torch.tensor(train_data_x_reshaped),torch.tensor(train_data_y_reshaped_1)) 
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True) 

valid_dataset = TensorDataset(torch.tensor(val_x_reshaped),torch.tensor(val_y_reshaped_1))
valid_dataloader = DataLoader(valid_dataset, batch_size=128, shuffle=True)

test_dataset = TensorDataset(torch.tensor(test_data_x_reshaped),torch.tensor(test_data_y_reshaped_1)) 
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True) 

In [125]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [126]:
sequence_length = 24 # 
feature_size = 114 # 
hidden_size = 128 # Hidden Layer 사이즈 설정처럼 설정
num_layers = 4 # stacked RNN (최대 4개까지는 Gradient Vanishing 현상이 적을 수 있으므로)
dropout_p = 0.2 # dropout rate
output_size = 4 # 
#minibatch_size = 128 # minibatch_size

In [127]:
class Net(nn.Module):
    def __init__(self, feature_size, hidden_size, num_layers, dropout_p, output_size, model_type):
        super().__init__()
        if model_type == 'rnn':
            self.sequenceclassifier = nn.RNN(
                input_size = feature_size,
                hidden_size = hidden_size,
                num_layers = num_layers,
                batch_first = True,
                dropout = dropout_p,
                bidirectional = True
            )
        elif model_type == 'lstm':
            self.sequenceclassifier = nn.LSTM(
                input_size = feature_size,
                hidden_size = hidden_size,
                num_layers = num_layers,
                batch_first = True,
                dropout = dropout_p,
                bidirectional = True
            )
        self.layers = nn.Sequential(
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(hidden_size * 2),
            # self.rnn() 의 출력값은 (batch_size, sequence_length, bidirections * hidden_size)
            # bidirectional 이 True 이므로, bidirections 는 2, 즉 2 * hidden_size
            nn.Linear(hidden_size * 2, output_size),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, x):
        # |x| = batch_first=True 이므로 (batch_size, sequence_length, input_size)
        out, _ = self.sequenceclassifier(x) # output, h_n 이므로, h_n 은 사용안함
        # output, h_n 이므로, h_n 은 사용안함
        # |out| = batch_first=True 이므로 (batch_size, sequence_length, 2 * hidden_size)
        # bidirectional 이 True 이면, bidirections 는 2 * hidden_size
        out = out[:, -1]
        # out[:, -1] 은 (batch_size, sequence_length, 2 * hidden_size) 에서, 
        # 전체 batch_size 를 선택한다는 의미가 :, 
        # sequence_length 인 28개의 순서가 있고, 각 순서마다 2 * hidden_size 만큼 있음
        # 이중에 최종 값은 맨 마지막  sequence_length 의 2 * hidden_size 임
        # |out| = (batch_size, hidden_size * 2)
        y = self.layers(out)
        # |y| = (batch_size, output_size)
        return y

In [128]:
model = Net(feature_size, hidden_size, num_layers, dropout_p, output_size, 'lstm').to(device)
model

Net(
  (sequenceclassifier): LSTM(114, 128, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
  (layers): Sequential(
    (0): LeakyReLU(negative_slope=0.1)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=256, out_features=4, bias=True)
    (3): LogSoftmax(dim=-1)
  )
)

In [129]:
from copy import deepcopy

loss_func = nn.NLLLoss() # log softmax 는 NLLLoss() 로 진행해야 함
optimizer = torch.optim.Adam(model.parameters()) # Adam, learning rate 필요없음

def train_model(model, early_stop, n_epochs, progress_interval):
    
    train_losses, valid_losses, lowest_loss = list(), list(), np.inf

    for epoch in range(n_epochs):
        
        train_loss, valid_loss = 0, 0
        
        # train the model
        model.train() # prep model for training
        for x_minibatch, y_minibatch in train_dataloader:
            x_minibatch, y_minibatch = x_minibatch.to(device), y_minibatch.to(device) 
            #y_minibatch = y_minibatch.to(device)    
            #mask = torch.sum(x_minibatch, axis = 1)
            #mask[mask>0] = 1
            y_minibatch_pred = model(x_minibatch)
            y_minibatch_indices = torch.argmax(y_minibatch, dim=1) 
            loss = loss_func(y_minibatch_pred , y_minibatch_indices )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss = train_loss / len(train_dataloader)
        train_losses.append(train_loss)      
        
        # validate the model
        model.eval()
        with torch.no_grad():
            for x_minibatch, y_minibatch in valid_dataloader:
                x_minibatch, y_minibatch = x_minibatch.to(device), y_minibatch.to(device) 
                #x_minibatch = x_minibatch.to(device)
                #y_minibatch = y_minibatch.to(device)      
                #mask = torch.sum(x_minibatch, axis = 1)
                #mask[mask>0] = 1
                y_minibatch_pred = model(x_minibatch)
                y_minibatch_indices = torch.argmax(y_minibatch, dim=1) 
                loss = loss_func(y_minibatch_pred , y_minibatch_indices )
                #loss = loss_func(y_minibatch_pred , y_minibatch )
                valid_loss += loss.item()
                
        valid_loss = valid_loss / len(valid_dataloader)
        valid_losses.append(valid_loss)

        if valid_losses[-1] < lowest_loss:
            lowest_loss = valid_losses[-1]
            lowest_epoch = epoch
            best_model = deepcopy(model.state_dict())
        else:
            if (early_stop > 0) and lowest_epoch + early_stop < epoch:
                print ("Early Stopped", epoch, "epochs")
                break
                
        if (epoch % progress_interval) == 0:
            print (train_losses[-1], valid_losses[-1], lowest_loss, lowest_epoch, epoch)
            
    model.load_state_dict(best_model)        
    return model, lowest_loss, train_losses, valid_losses

nb_epochs = 100 
progress_interval = 3
early_stop = 30

model, lowest_loss, train_losses, valid_losses = train_model(model, early_stop, nb_epochs, progress_interval)

0.5593864692237016 1.0494960476743413 1.0494960476743413 0 0
0.4928057258738611 0.9434687996164282 0.4810644044203961 1 3
0.489578503014962 0.671127736251405 0.4760943524380948 4 6
0.4877077256890552 0.4731477181962196 0.4731477181962196 9 9
0.48673165047783346 0.5648033070437452 0.4731477181962196 9 12
0.4852316913011891 0.5658165308389258 0.4731477181962196 9 15
0.4850455620010993 0.6490441801700186 0.4707200051622188 16 18
0.4843041770536328 0.47334982645004353 0.4707200051622188 16 21
0.4850391830236648 0.47747777687742354 0.4707200051622188 16 24
0.48402888343042255 0.6754174765120161 0.4707200051622188 16 27
0.4840970302018648 0.9334656038182847 0.4707200051622188 16 30
0.4846900594285684 0.5889445181856764 0.4707200051622188 16 33
0.4844467827988234 0.48475199050091683 0.4707200051622188 16 36
0.48404937993763086 0.4934372435858909 0.4707200051622188 16 39
0.48281675335479984 0.5095708611163687 0.4707200051622188 16 42
0.48160149272287184 0.4641695133549102 0.4641695133549102 45

In [130]:
test_loss = 0
correct = 0
wrong_samples, wrong_preds, actual_preds = list(), list(), list()

model.eval()
with torch.no_grad():
    for x_minibatch, y_minibatch in test_dataloader:
        x_minibatch = x_minibatch.to(device)  
        y_minibatch = y_minibatch.to(device)      
        y_test_pred = model(x_minibatch)
        y_minibatch_indices = torch.argmax(y_minibatch, dim=1) 
        test_loss += loss_func(y_test_pred, y_minibatch_indices)  
        pred = torch.argmax(y_test_pred, dim=1)
        correct += pred.eq(y_minibatch_indices).sum().item()
        
        wrong_idx = pred.ne(y_minibatch_indices).nonzero()[:, 0].cpu().numpy().tolist()
        for index in wrong_idx:
            wrong_samples.append(x_minibatch[index].cpu())
            wrong_preds.append(pred[index].cpu())
            actual_preds.append(y_minibatch[index].cpu())
            
test_loss /= len(test_dataloader.dataset)
print('Average Test Loss: {:.4f}'.format( test_loss ))
print('Accuracy: {}/{} ({:.2f}%)'.format( correct, len(test_dataloader.dataset), 100 * correct / len(test_dataloader.dataset) ))

Average Test Loss: 0.0037
Accuracy: 37816/47661 (79.34%)


In [131]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# ...

test_loss = 0
correct = 0
wrong_samples, wrong_preds, actual_preds = list(), list(), list()

model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for x_minibatch, y_minibatch in test_dataloader:
        x_minibatch = x_minibatch.to(device)  
        y_minibatch = y_minibatch.to(device)      
        y_test_pred = model(x_minibatch)
        y_minibatch_indices = torch.argmax(y_minibatch, dim=1) 
        test_loss += loss_func(y_test_pred, y_minibatch_indices)  
        pred = torch.argmax(y_test_pred, dim=1)
        correct += pred.eq(y_minibatch_indices).sum().item()

        all_preds.extend(pred.cpu().numpy())
        all_targets.extend(y_minibatch_indices.cpu().numpy())

        wrong_idx = pred.ne(y_minibatch_indices).nonzero()[:, 0].cpu().numpy().tolist()
        for index in wrong_idx:
            wrong_samples.append(x_minibatch[index].cpu())
            wrong_preds.append(pred[index].cpu())
            actual_preds.append(y_minibatch[index].cpu())

test_loss /= len(test_dataloader.dataset)

# Calculate precision, recall, and confusion matrix
precision = precision_score(all_targets, all_preds, average='weighted')
recall = recall_score(all_targets, all_preds, average='weighted')
conf_matrix = confusion_matrix(all_targets, all_preds)

print('Average Test Loss: {:.4f}'.format(test_loss))
print('Accuracy: {}/{} ({:.2f}%)'.format(correct, len(test_dataloader.dataset), 100 * correct / len(test_dataloader.dataset)))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('Confusion Matrix:\n', conf_matrix)


Average Test Loss: 0.0037
Accuracy: 37816/47661 (79.34%)
Precision: 0.7856
Recall: 0.7934
Confusion Matrix:
 [[26167  3211    13]
 [ 5058 11236   312]
 [  134  1117   413]]
